# FinUties API — Quickstart

Connect to **data.finuties.com** in under 5 minutes.  
This notebook walks through every step: health check → discover data → authenticate → query.

| Endpoint | URL |
|---|---|
| Base | `https://data.finuties.com` |
| Docs | `https://data.finuties.com` (browser) |
| GitHub | [techuties/finuties](https://github.com/techuties/finuties) |

> **Prerequisites:** `pip install requests pandas` (both are standard; no exotic deps).

In [ ]:
import requests, json, os
import pandas as pd

BASE = os.getenv("FINUTIES_API_URL", "https://data.finuties.com")

def api(method, path, **kw):
    """Tiny helper — adds base URL + auth header when a token exists."""
    headers = kw.pop("headers", {})
    headers["Accept"] = "application/json"
    if TOKEN:
        headers["Authorization"] = f"Bearer {TOKEN}"
    resp = requests.request(method, f"{BASE}{path}", headers=headers, **kw)
    resp.raise_for_status()
    return resp.json()

TOKEN = None  # set after login
print(f"Target: {BASE}")

---
## 1 · Health Check (no auth required)
Confirm the API is online before doing anything else.

In [ ]:
health = api("GET", "/health")
print(health)  # → {"status": "online"}

---
## 2 · Discover Available Data (no auth required)
The `/api/v1/resources` endpoint returns **every domain and resource** available.

In [ ]:
catalog = api("GET", "/api/v1/resources")

# Pretty-print domains
for domain, info in catalog["domains"].items():
    resources = info["resources"]
    print(f"  {domain:14s}  ({len(resources):2d})  {info['description'][:60]}")

total = sum(len(v["resources"]) for v in catalog["domains"].values())
print(f"\n  Total resources: {total}")

---
## 3 · Authenticate
Two options — pick one:

| Method | When to use |
|---|---|
| **JWT** (login) | Interactive / notebook sessions |
| **API key** (`fin_sk_…`) | Scripts, CI/CD, long-running jobs |

### Option A — Login with username & password

In [ ]:
# ⬇ Replace with your credentials (sign up at https://www.finuties.com)
USERNAME = "your_username"
PASSWORD = "your_password"

resp = requests.post(f"{BASE}/api/v1/auth/login", json={
    "username": USERNAME,
    "password": PASSWORD,
})

if resp.ok:
    data = resp.json()
    TOKEN = data["access_token"]
    print(f"Logged in as {data['username']} (role: {data['role']})")
    print(f"Token: {TOKEN[:20]}…")
else:
    print(f"Login failed: {resp.status_code} — {resp.text}")

### Option B — Use an API key
API keys are created from the platform settings or via `POST /api/v1/auth/api-keys`.

In [ ]:
# Uncomment and paste your API key:
# TOKEN = "fin_sk_your_api_key_here"

---
## 4 · Query Data — GET (simple)
The pattern is always:
```
GET /api/v1/{domain}/{resource}?filter=value&limit=N
```
Let's fetch the 5 most recent SEC filings.

In [ ]:
filings = api("GET", "/api/v1/sec/filings", params={
    "limit": 5,
    "order_dir": "desc",
})

df = pd.DataFrame(filings["items"])
print(f"Returned {len(df)} rows  |  cached: {filings.get('cached', False)}")
df[["cik", "form_type", "filed_at"]].head()

---
## 5 · Query Data — POST (advanced filters)
Use `POST /api/v1/query` for complex queries with ranges, IN-lists, and pagination.

**Example:** Latest stock prices for a few symbols.

In [ ]:
result = api("POST", "/api/v1/query", json={
    "domain":     "market",
    "resource":   "stock-prices",
    "in_filters": {"symbol": ["AAPL", "MSFT", "GOOG"]},
    "order_by":   "date",
    "order_dir":  "desc",
    "limit":      30,
    "include_total": True,
})

df = pd.DataFrame(result["items"])
print(f"{result['total']} total rows  |  showing {len(df)}  |  cached: {result.get('cached', False)}")
df[["symbol", "date", "open", "close", "volume"]].head(10)

---
## 6 · Pagination
Paginate through large datasets with `limit` + `offset`.

In [ ]:
all_rows = []
offset = 0
PAGE = 200  # max per request

for page_num in range(3):  # cap at 3 pages for this demo
    batch = api("GET", "/api/v1/sec/filings", params={
        "limit": PAGE, "offset": offset, "order_dir": "desc",
    })
    items = batch["items"]
    all_rows.extend(items)
    print(f"  page {page_num+1}: {len(items)} rows (offset={offset})")
    if len(items) < PAGE:
        break  # last page
    offset += PAGE

df_all = pd.DataFrame(all_rows)
print(f"\nCollected {len(df_all)} rows total")

---
## 7 · Error Handling
Common status codes and what they mean.

In [ ]:
# Intentional error — query without auth to see the response shape
resp = requests.get(f"{BASE}/api/v1/sec/filings", headers={"Accept": "application/json"})
print(f"Status: {resp.status_code}")
print(f"Body:   {resp.json()}")

print("\n--- Common status codes ---")
codes = [
    (200, "Success"),
    (401, "Missing or invalid token"),
    (402, "Insufficient token balance — deposit at terminal.finuties.com/settings"),
    (404, "Resource not found"),
    (429, "Rate limit exceeded — wait and retry"),
    (503, "Service temporarily unavailable"),
]
for code, desc in codes:
    print(f"  {code}  {desc}")

---
## 8 · Quick Reference

| Action | Method | Endpoint | Auth? |
|---|---|---|---|
| Health check | `GET` | `/health` | No |
| Resource catalog | `GET` | `/api/v1/resources` | No |
| Login | `POST` | `/api/v1/auth/login` | No |
| Verify token | `GET` | `/api/v1/auth/verify` | Yes |
| User info | `GET` | `/api/v1/auth/me` | Yes |
| Simple query | `GET` | `/api/v1/{domain}/{resource}` | Yes |
| Advanced query | `POST` | `/api/v1/query` | Yes |
| SEC filings (latest) | `GET` | `/api/v1/sec/filings/latest` | Yes |

**Query parameters** (GET): `limit`, `offset`, `order_by`, `order_dir`, `include_total`, `no_cache`, plus any column filter.  
**Body fields** (POST /query): `domain`, `resource`, `filters`, `in_filters`, `ranges`, `limit`, `offset`, `order_by`, `order_dir`, `include_total`.

---

*FinUties — financial data, unified.*  
Feedback? Open an issue at [github.com/techuties/finuties](https://github.com/techuties/finuties).